In [35]:
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementNotInteractableException
from dotenv import load_dotenv

load_dotenv(override=True)

options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)  # Mantém o navegador aberto

service = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=service, options=options)

# Função para resolver pop-ups que podem travar o código
def fechar_popup():
    try:
        modal = navegador.find_element(By.CLASS_NAME, 'artdeco-modal__dismiss')
        modal.click()  # Fecha pop-up de confirmação
    except:
        pass  # Ignora erro se o pop-up não existir

# Função para realizar login no LinkedIn
def login():
    navegador.get('https://www.linkedin.com/login/pt?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')  # Acessa a página de login
    WebDriverWait(navegador, 10).until(EC.presence_of_element_located((By.ID, "username")))
    navegador.find_element(By.ID, "username").send_keys(os.getenv("LINKEDIN_EMAIL"))# Preenche login
    navegador.find_element(By.ID, "password").send_keys(os.getenv("LINKEDIN_PASSWORD"))  # Preenche senha
    navegador.find_element(By.XPATH, '//*[@id="organic-div"]/form/div[4]/button').click()  # Clica em "Entrar"
    time.sleep(3)  # Pequena pausa para garantir login

# Função para buscar Tech Recruiters
def buscar_recruiters():
    search_bar = WebDriverWait(navegador, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'search-global-typeahead__input')))
    search_bar.send_keys('Tech Recruiter', Keys.ENTER)  # Pesquisa "Tech Recruiter"
    time.sleep(3)  # Aguarda carregamento da página

    # Seleciona o filtro "Pessoas"
    filtros = navegador.find_elements(By.CLASS_NAME, 'search-reusables__primary-filter')
    for filtro in filtros:
        if filtro.text == 'Pessoas':
            filtro.click()
            break
    time.sleep(3)  # Aguarda a página atualizar

# Função para enviar solicitação de conexão com mensagem personalizada

def enviar_conexoes():
    delay = 30  # Tempo máximo de espera para encontrar elementos
    conta_conexoes = 0  # Contador de conexões enviadas
    mensagem = "Td bem? Sou Gabriel, dev com 3,5 anos de experiência em Python e back-end. Busco vaga Pleno. Pretensão: R$5.500 Podemos falar a respeito?"

    while conta_conexoes < 50:  # Continua até enviar 20 conexões
        try:
            # Busca resultados da pesquisa
            #lista_resultados = navegador.find_elements(By.CLASS_NAME, 'hOppSLaLUJmzZeoBPionNQNf') # sempre muda verificar clicando na cabeça da pessoa com ispecionar e pegando a caixa completa com todos os campos e o classname do cmpo completo
            lista_resultados = navegador.find_elements(By.XPATH, "//*[contains(@data-chameleon-result-urn, 'urn:li:member')]") # aqui pego por outra div que pelo xpath que não muda muito pesquiso dentro do texto se contem o que a variavel diz
            for resultado in lista_resultados:
                try:
                    nome_recrutador = WebDriverWait(resultado, delay).until(
                        EC.presence_of_element_located((By.CLASS_NAME, 't-sans'))).text.split()[0]
                    botao_conectar = WebDriverWait(resultado, delay).until(
                        EC.presence_of_element_located((By.CLASS_NAME, 'artdeco-button__text')))
                    
                    if botao_conectar.text == 'Conectar':
                        conta_conexoes += 1  # Aumenta contador
                        botao_conectar.click()  # Clica no botão "Conectar"
                        time.sleep(2)
                        
                        botoes_tela = navegador.find_elements(By.CLASS_NAME, 'artdeco-button__text')
                        for botao in botoes_tela:
                            if botao.text == "Adicionar nota":
                                botao.click()
                                break
                        time.sleep(2)
                        # Insere mensagem personalizada
                        navegador.find_element(By.CLASS_NAME, 'ember-text-area').send_keys(f"Olá {nome_recrutador}. " + mensagem)
                        time.sleep(2)
                        
                        # Envia mensagem
                        navegador.find_element(By.CLASS_NAME, 'ml1').click()
                        time.sleep(2)
                        
                        fechar_popup()  # Fecha pop-up caso apareça
                    
                except ElementNotInteractableException:
                    fechar_popup()  # Fecha pop-up se algo travar

            # Rola a página para carregar mais resultados
            time.sleep(2)
            navegador.execute_script("window.scroll(0, 1300)")
            time.sleep(1)
            
            # Avança para a próxima página
            avancar = WebDriverWait(navegador, delay).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'artdeco-pagination__button--next')))
            avancar.click()
            time.sleep(5)
            navegador.refresh()
            time.sleep(5)
            
        except TimeoutException:
            print("Tempo limite atingido. Finalizando a automação...")
            break

# Executando funções em ordem correta
login()
buscar_recruiters()
enviar_conexoes()

# Fecha o navegador ao final
navegador.quit()

In [37]:
# Função para resolver pop-ups que podem travar o código
def fechar_popup():
    try:
        modal = navegador.find_element(By.CLASS_NAME, 'artdeco-modal__dismiss')
        modal.click()  # Fecha pop-up de confirmação
    except:
        pass  # Ignora erro se o pop-up não existir

# Função para realizar login no LinkedIn
def login():
    navegador.get('https://www.linkedin.com/login/pt?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')  # Acessa a página de login
    WebDriverWait(navegador, 10).until(EC.presence_of_element_located((By.ID, "username")))
    navegador.find_element(By.ID, "username").send_keys(os.getenv("LINKEDIN_EMAIL"))# Preenche login
    navegador.find_element(By.ID, "password").send_keys(os.getenv("LINKEDIN_PASSWORD"))  # Preenche senha
    navegador.find_element(By.XPATH, '//*[@id="organic-div"]/form/div[4]/button').click()  # Clica em "Entrar"
    time.sleep(3)  # Pequena pausa para garantir login

# Função para buscar Tech Recruiters
def buscar_recruiters():
    search_bar = WebDriverWait(navegador, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'search-global-typeahead__input')))
    search_bar.send_keys('Tech Recruiter', Keys.ENTER)  # Pesquisa "Tech Recruiter"
    time.sleep(3)  # Aguarda carregamento da página

    # Seleciona o filtro "Pessoas"
    filtros = navegador.find_elements(By.CLASS_NAME, 'search-reusables__primary-filter')
    for filtro in filtros:
        if filtro.text == 'Pessoas':
            filtro.click()
            break
    time.sleep(3)  # Aguarda a página atualizar

# Função para enviar solicitação de conexão com mensagem personalizada

def enviar_conexoes():
    delay = 30  # Tempo máximo de espera para encontrar elementos
    conta_conexoes = 0  # Contador de conexões enviadas
    mensagem = "Td bem? Sou Gabriel, dev com 3,5 anos de experiência em Python e back-end. Busco vaga Pleno. Pretensão: R$5.500 Podemos falar a respeito?"

    while conta_conexoes < 50:  # Continua até enviar 20 conexões
        try:
            # Busca resultados da pesquisa
            #lista_resultados = navegador.find_elements(By.CLASS_NAME, 'hOppSLaLUJmzZeoBPionNQNf') # sempre muda verificar clicando na cabeça da pessoa com ispecionar e pegando a caixa completa com todos os campos e o classname do cmpo completo
            lista_resultados = navegador.find_elements(By.XPATH, "//*[contains(@data-chameleon-result-urn, 'urn:li:member')]") # aqui pego por outra div que pelo xpath que não muda muito pesquiso dentro do texto se contem o que a variavel diz
            for resultado in lista_resultados:
                try:
                    nome_recrutador = WebDriverWait(resultado, delay).until(
                        EC.presence_of_element_located((By.CLASS_NAME, 't-sans'))).text.split()[0]
                    botao_conectar = WebDriverWait(resultado, delay).until(
                        EC.presence_of_element_located((By.CLASS_NAME, 'artdeco-button__text')))
                    
                    if botao_conectar.text == 'Conectar':
                        conta_conexoes += 1  # Aumenta contador
                        botao_conectar.click()  # Clica no botão "Conectar"
                        time.sleep(2)
                        
                        botoes_tela = navegador.find_elements(By.CLASS_NAME, 'artdeco-button__text')
                        for botao in botoes_tela:
                            if botao.text == "Adicionar nota":
                                botao.click()
                                break
                        time.sleep(2)
                        # Insere mensagem personalizada
                        navegador.find_element(By.CLASS_NAME, 'ember-text-area').send_keys(f"Olá {nome_recrutador}. " + mensagem)
                        time.sleep(2)
                        
                        # Envia mensagem
                        navegador.find_element(By.CLASS_NAME, 'ml1').click()
                        time.sleep(2)
                        
                        fechar_popup()  # Fecha pop-up caso apareça
                    
                except ElementNotInteractableException:
                    fechar_popup()  # Fecha pop-up se algo travar

            # Rola a página para carregar mais resultados
            time.sleep(2)
            navegador.execute_script("window.scroll(0, 1300)")
            time.sleep(1)
            
            # Avança para a próxima página
            avancar = WebDriverWait(navegador, delay).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'artdeco-pagination__button--next')))
            avancar.click()
            time.sleep(5)
            navegador.refresh()
            time.sleep(5)
            
        except TimeoutException:
            print("Tempo limite atingido. Finalizando a automação...")
            break

# Executando funções em ordem correta
login()
buscar_recruiters()
enviar_conexoes()

# Fecha o navegador ao final
navegador.quit()

KeyboardInterrupt: 